# Data Pre-processing

# **RQ2. 따릉이 대여권 유형(일일권, 정기권 등)에 영향을 미치는 요소에는 무엇이 있을까?**

In [6]:
import numpy as np
import pandas as pd
import scipy.stats

In [8]:
import os, sys 
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [9]:
cd '/content/gdrive/My Drive/Data Mining/실습/실습2'


/content/gdrive/My Drive/Data Mining/실습/실습2


In [10]:
ls -l

total 48660
-rw------- 1 root root 48868357 Mar 16 02:56 '공공자전거 이용정보(일별)_2021.01.csv'
-rw------- 1 root root   630937 Mar 16 03:51  2021.03.16._DM실습2.html
-rw------- 1 root root   205948 Mar 18 17:01 '공공자전거 대여소 정보(21.01.31 기준).csv'
-rw------- 1 root root    14950 Mar 16 10:46 '실습2(2021-03-16).ipynb'
-rw------- 1 root root      353 Mar 16 03:51  cars.csv
-rw------- 1 root root     2768 Mar 16 03:51  dm_ex2.csv
-rw------- 1 root root    72589 Mar 18 18:17  HW1_강수빈.ipynb
-rw------- 1 root root    28706 Mar 18 18:19  HW1_total.ipynb
-rw------- 1 root root       50 Mar 16 03:53  work_ex.txt


#**Read Data**

In [11]:
#2021년 1월 공공자전거 일별 이용정보
df_day=pd.read_csv(r'공공자전거 이용정보(일별)_2021.01.csv', encoding='cp949')
df_day=df_day.reset_index(inplace=False)
df_day

,index,대여일자,대여소번호,대여소,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분)
0,0,2021-01-01,101,101. (구)합정동 주민센터,일일(회원),\N,AGE_002,1,0.00,0.00,0.00,46
1,1,2021-01-01,101,101. (구)합정동 주민센터,일일(회원),M,AGE_002,1,0.00,0.00,0.00,48
2,2,2021-01-01,101,101. (구)합정동 주민센터,정기,\N,AGE_002,1,0.00,0.00,0.00,31
3,3,2021-01-01,101,101. (구)합정동 주민센터,정기,\N,AGE_005,1,0.00,0.00,0.00,30
4,4,2021-01-01,101,101. (구)합정동 주민센터,정기,F,AGE_003,1,0.00,0.00,0.00,78
...,...,...,...,...,...,...,...,...,...,...,...,...
474626,474626,2021-01-31,3587,3587.우성식품 앞,정기,F,AGE_003,2,42.03,0.40,1746.75,77
474627,474627,2021-01-31,3587,3587.우성식품 앞,정기,M,AGE_005,1,129.34,1.11,4803.09,41
474628,474628,2021-01-31,3588,3588.세종대학교(영실관),정기,M,AGE_002,2,33.56,0.28,1210.58,11
474629,474629,2021-01-31,3588,3588.세종대학교(영실관),정기,M,AGE_003,2,25.68,0.25,1080.61,11


In [12]:
#2021년 1월 기준 공공자전거 대여소 정보 
df_loc = pd.read_csv('공공자전거 대여소 정보(21.01.31 기준).csv', encoding='cp949')
# df_sdm_idx = df_loc[df_loc['소재지(위치)'] != '서대문구'].index
# df_sdm = df_loc.drop(df_sdm_idx)
# df_sdm['대여소\n번호'] = df_sdm['대여소\n번호'].astype(int)

# sdm_loc_list = df_sdm['대여소\n번호']
# sdm_loc_list
df_loc

,대여소번호,대여소명,자치구,상세주소,위도,경도,설치시기,거치대수(LCD),거치대수(QR),운영방식
0,301,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130,37.575794,126.971451,2015-10-07,16.0,NaN,LCD
1,302,경복궁역 4번출구 뒤,종로구,서울특별시 종로구 사직로 지하130,37.575947,126.974060,2015-10-07,12.0,NaN,LCD
2,303,광화문역 1번출구 앞,종로구,서울특별시 종로구 세종대로 지하189,37.571770,126.974663,2015-10-07,8.0,NaN,LCD
3,304,광화문역 2번출구 앞,종로구,서울특별시 종로구 세종대로 지하172,37.572113,126.977577,2021-01-26,NaN,7.0,QR
4,305,종로구청 옆,종로구,서울특별시 종로구 삼봉로 43,37.572582,126.978355,2015-10-07,16.0,NaN,LCD
...,...,...,...,...,...,...,...,...,...,...
2149,1085,래미안 솔베뉴(고덕로),강동구,양재대로1650,37.554569,127.145874,2020-01-13,NaN,20.0,QR
2150,1086,래미안 솔베뉴(구천면로),강동구,양재대로1650,37.550598,127.145454,2020-01-10,NaN,10.0,QR
2151,1088,풍납사거리(강동),강동구,성내동451-3,37.528122,127.119522,2020-01-14,NaN,12.0,QR
2152,1089,고덕 래미안힐스테이트(201동),강동구,고덕동 675,37.560600,127.146698,2020-01-13,NaN,10.0,QR


In [13]:
#일별 이용정보에 대여소 정보 추가
df_day_loc = pd.merge(df_day, df_loc, left_on='대여소번호', right_on='대여소번호', how='inner')
del_idx = df_day_loc[(df_day_loc['이용시간(분)']<=0) | (df_day_loc['이동거리(M)']<=0)].index
df_day_loc = df_day_loc.drop(del_idx) #이용시간이나 이동거리가 0이하일 경우 정확하지 않은 데이터라 판단하고 삭제
df_day_loc

,index,대여일자,대여소번호,대여소,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분),대여소명,자치구,상세주소,위도,경도,설치시기,거치대수(LCD),거치대수(QR),운영방식
5,5,2021-01-01,101,101. (구)합정동 주민센터,정기,M,AGE_003,1,36.23,0.29,1270.63,123,(구)합정동 주민센터,마포구,서울특별시 마포구 동교로8길 58,37.549561,126.905754,2015-09-06,5.0,NaN,LCD
7,14775,2021-01-02,101,101. (구)합정동 주민센터,일일(회원),\N,AGE_003,1,32.58,0.38,1645.60,13,(구)합정동 주민센터,마포구,서울특별시 마포구 동교로8길 58,37.549561,126.905754,2015-09-06,5.0,NaN,LCD
8,14776,2021-01-02,101,101. (구)합정동 주민센터,정기,\N,AGE_002,3,38.36,0.39,1699.35,72,(구)합정동 주민센터,마포구,서울특별시 마포구 동교로8길 58,37.549561,126.905754,2015-09-06,5.0,NaN,LCD
10,14778,2021-01-02,101,101. (구)합정동 주민센터,정기,\N,AGE_005,3,16.40,0.15,637.23,114,(구)합정동 주민센터,마포구,서울특별시 마포구 동교로8길 58,37.549561,126.905754,2015-09-06,5.0,NaN,LCD
11,14779,2021-01-02,101,101. (구)합정동 주민센터,정기,F,AGE_003,1,23.46,0.21,911.44,11,(구)합정동 주민센터,마포구,서울특별시 마포구 동교로8길 58,37.549561,126.905754,2015-09-06,5.0,NaN,LCD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475073,466253,2021-01-31,1627,1627. 수락산역 4번출구,일일(회원),M,AGE_002,1,154.46,1.13,4875.63,31,수락산역 4번출구,노원구,서울특별시 노원구 동일로 지하 1662,37.676941,127.055099,2020-12-31,NaN,12.0,QR
475075,466255,2021-01-31,1627,1627. 수락산역 4번출구,정기,F,AGE_002,1,72.19,0.70,3038.28,32,수락산역 4번출구,노원구,서울특별시 노원구 동일로 지하 1662,37.676941,127.055099,2020-12-31,NaN,12.0,QR
475076,466256,2021-01-31,1627,1627. 수락산역 4번출구,정기,F,AGE_004,1,100.47,1.01,4374.36,23,수락산역 4번출구,노원구,서울특별시 노원구 동일로 지하 1662,37.676941,127.055099,2020-12-31,NaN,12.0,QR
475077,466257,2021-01-31,1627,1627. 수락산역 4번출구,정기,F,AGE_005,1,74.00,0.62,2669.48,36,수락산역 4번출구,노원구,서울특별시 노원구 동일로 지하 1662,37.676941,127.055099,2020-12-31,NaN,12.0,QR


#**Preprocessing X and Y & Result**

### 1) 이용권과 연령대는 관련이 있을까?

In [14]:
ticket_age=df_day_loc.pivot_table(index='대여구분코드',columns='연령대코드',values='index', aggfunc='count')
ticket_age=ticket_age.drop(index=["BIL_021"])
ticket_age=ticket_age.fillna(0)
ticket_age

연령대코드,AGE_001,AGE_002,AGE_003,AGE_004,AGE_005,AGE_006,AGE_007,AGE_008
대여구분코드,,,,,,,,
단체,588.0,725.0,305.0,1030.0,175.0,17.0,2.0,42.0
일일(비회원),0.0,1.0,4.0,0.0,0.0,0.0,0.0,2200.0
일일(회원),9023.0,43222.0,21870.0,11293.0,4327.0,852.0,167.0,1464.0
정기,11416.0,98799.0,81926.0,66735.0,50795.0,19829.0,3107.0,4951.0


### 2) 이용권과 이동거리는 관련이 있을까?

이동 거리는 continuous한 데이터이기 때문에 1km 단위로 범주화(이산화)시킨다.

In [15]:
df_day_loc['Distance_cut']=pd.cut(df_day_loc["이동거리(M)"], bins=[0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,463301.420000], labels=['~1km','~2km','~3km','~4km','~5km','~6km','~7km','~8km','~9km','~10km','10km~'])
ticket_dist=df_day_loc.pivot_table(index='대여구분코드',columns='Distance_cut',values='index', aggfunc='count')
ticket_dist=ticket_dist.drop(index=["BIL_021"])
ticket_dist=ticket_dist.fillna(0)
ticket_dist

Distance_cut,~1km,~2km,~3km,~4km,~5km,~6km,~7km,~8km,~9km,~10km,10km~
대여구분코드,,,,,,,,,,,
단체,70.0,187.0,215.0,209.0,210.0,203.0,192.0,154.0,156.0,136.0,1152.0
일일(비회원),138.0,263.0,300.0,232.0,184.0,146.0,142.0,117.0,83.0,66.0,534.0
일일(회원),9645.0,16901.0,12621.0,9314.0,6994.0,5412.0,4455.0,3668.0,3066.0,2490.0,17652.0
정기,60173.0,74800.0,49007.0,33358.0,24505.0,17693.0,13446.0,10948.0,8571.0,6909.0,38148.0


### 3) 이용권과 이동시간은 관련이 있을까?

이동 시간는 continuous한 데이터이기 때문에 10개의 범주로 나눈다. 

In [16]:
df_day_loc['Time_cut'] = pd.qcut(df_day_loc['이용시간(분)'], 10, labels=['~6분', '~9분', '~13분', '~19분', '~26분', '~36분','~49분','~68분','~105분', '105분~'])
ticket_time=df_day_loc.pivot_table(index='대여구분코드',columns='Time_cut',values='index', aggfunc='count')
ticket_time=ticket_time.drop(index=["BIL_021"])
ticket_time=ticket_time.fillna(0)
ticket_time

Time_cut,~6분,~9분,~13분,~19분,~26분,~36분,~49분,~68분,~105분,105분~
대여구분코드,,,,,,,,,,
단체,19,19,54,79,123,185,253,405,672,1075
일일(비회원),61,57,113,149,184,245,293,350,346,407
일일(회원),5960,5868,7552,9134,8391,9606,10529,11320,10733,13125
정기,45740,30971,34127,38819,33111,33222,31203,30191,31710,28464


###4) 이용권과 성별은 관련이 있을까?

In [17]:
#성별 전처리 - '\n'값은 삭제하고 성별의 명칭을 통일해준다.
del_idx = df_day_loc[df_day_loc['성별'] == "\\N"].index
df_gender = df_day_loc.drop(del_idx) 
df_gender = df_gender.replace('f','F', regex=True)
df_gender = df_gender.replace('m','M', regex=True)
ticket_gender=df_gender.pivot_table(index='대여구분코드',columns='성별',values='index', aggfunc='count')
ticket_gender=ticket_gender.drop(index=["BIL_021"])
ticket_gender=ticket_gender.fillna(0)
ticket_gender

성별,F,M
대여구분코드,,
단체,678,835
일일(비회원),32,19
일일(회원),22335,28707
정기,67957,121254


### 5) 이용권과 대여소의 운영방식은 관련이 있을까?

In [18]:
ticket_oper=df_day_loc.pivot_table(index='대여구분코드',columns='운영방식',values='index', aggfunc='count')
ticket_oper=ticket_oper.drop(index=["BIL_021"])
ticket_oper=ticket_oper.fillna(0)
ticket_oper

운영방식,LCD,QR
대여구분코드,,
단체,2236,648
일일(비회원),1704,501
일일(회원),69930,22288
정기,256618,80940


# **Result**

In [19]:
age_chi2, age_pvalue, age_dof, age_expected = scipy.stats.chi2_contingency(ticket_age)
dist_chi2, dist_pvalue, dist_dof, dist_expected = scipy.stats.chi2_contingency(ticket_dist)
time_chi2, time_pvalue, time_dof, time_expected = scipy.stats.chi2_contingency(ticket_time)
gender_chi2, gender_pvalue, gender_dof, gender_expected = scipy.stats.chi2_contingency(ticket_gender)
oper_chi2, oper_pvalue, oper_dof, oper_expected = scipy.stats.chi2_contingency(ticket_oper)

In [20]:
print("'이용권-연령대'간 카이제곱검정으로부터의 chi2: %.3f" % age_chi2)
print("'이용권-이동거리'간 카이제곱검정으로부터의 chi2: %.3f" % dist_chi2)
print("'이용권-이동시간'간 카이제곱검정으로부터의 chi2: %.3f" % time_chi2)
print("'이용권-성별'간 카이제곱검정으로부터의 chi2: %.3f" % gender_chi2)
print("'이용권-운영방식'간 카이제곱검정으로부터의 chi2: %.3f" % oper_chi2)

'이용권-연령대'간 카이제곱검정으로부터의 chi2: 136653.531
'이용권-이동거리'간 카이제곱검정으로부터의 chi2: 10033.340
'이용권-이동시간'간 카이제곱검정으로부터의 chi2: 12718.605
'이용권-성별'간 카이제곱검정으로부터의 chi2: 1100.478
'이용권-운영방식'간 카이제곱검정으로부터의 chi2: 7.212


In [21]:
print("'이용권-연령대'간 카이제곱검정으로부터의 pvalue: %.3f" % age_pvalue)
print("'이용권-이동거리'간 카이제곱검정으로부터의 pvalue: %.3f" % dist_pvalue)
print("'이용권-이동시간'간 카이제곱검정으로부터의 pvalue: %.3f" % time_pvalue)
print("'이용권-성별'간 카이제곱검정으로부터의 pvalue: %.3f" % gender_pvalue)
print("'이용권-운영방식'간 카이제곱검정으로부터의 pvalue: %.3f" % oper_pvalue)

'이용권-연령대'간 카이제곱검정으로부터의 pvalue: 0.000
'이용권-이동거리'간 카이제곱검정으로부터의 pvalue: 0.000
'이용권-이동시간'간 카이제곱검정으로부터의 pvalue: 0.000
'이용권-성별'간 카이제곱검정으로부터의 pvalue: 0.000
'이용권-운영방식'간 카이제곱검정으로부터의 pvalue: 0.065
